# Section 2

## Stange's order finding algorithm

In [ ]:
import numpy as np
def Stange_method(n,g,b,c=10):    

    #-------------------phase 1: Relation finding------------------------- 
    counter= 0               
    rand_x=[]                          
    FT=matrix(b,b+c,sparse=True) 

    while counter<b+c :
        x=randint(1,n)
        while x in rand_x:
            x=randint(1,n)
        residue = ZZ(Mod(g,n)^x)
        f=smoothness(residue,b)
        #------------------------------Append the vector if the relationship is found---------------------
        if f:
            f=vector(f)
            rand_x.append(x)
            FT[:,counter]=f
            counter+=1
    #------------------------------phase 2: Linear Algebra---------------------------------------

    kernel_basis=FT.right_kernel().basis()[:c] # obtain the basis
    
    #----------------------phase 3: GCD computation----------------------------

    alpha=[]
    for base_vec in kernel_basis:
        alp=np.dot(base_vec,rand_x) 
        alpha.append(alp)  #list of all the alphas

    order=Integer(gcd(alpha)) 
    return order

def smoothness(residue,b):
    f=[]
    p=2 
    i=0
    while residue!=1 and i<b:
        exponent=0
        while p.divides(residue):
            residue=residue/p           
            exponent+=1
        f.append(exponent)
        p=next_prime(p)
        i+=1
    g=False
    if residue==1:
        g=vector(ZZ,b)
        g[:len(f)]=f
    return g



n=765
g=11
b=10
if gcd(g,n)==1:
    print("Computed order of",g,"mod",n,"is",Stange_method(n,g,b,c=10))
    print("Correct order is",Mod(g,n).multiplicative_order())

# Section 3

## Computation of $b_n$

In [ ]:

def opt_b(n):
    C_n=sqrt(2 /  ( ln(n) * ln(ln(n)) )  )
    return ceil(C_n*exp(1/C_n))


## Testing over b

In [ ]:
from time import time
import numpy as np
import matplotlib.pyplot as plt

def test_optimal_b(n,g,bounds,y=5):
    times=[]
    for b in range(bounds[0],bounds[1]):
        summ=0
        for i in range(y):
            tock=time()
            Stange_method(n,g,b)
            tick=time()
            summ+=tick-tock
        summ/=5
        times.append(summ)
    return range(bounds[0],bounds[1]),times


In [ ]:
Integers=[537, 765, 8585, 19053, 61453, 101371, 199989, 247251, 484957, 671015, 871933, 907423, 1744953, 1946725, 2177645, 2381625, 3632763, 5001635, 7865455, 22653803]

for h in Integers:
    b=opt_b(h)
    if b>15:
        bounds=[8,50]
    else:
        bounds=[2,50]
    g=randint(2,h-1)
    while gcd(g,h)!=1:
        g=randint(2,h-2)
        
    res1,res2=test_optimal_b(h,g,bounds,5) 
    
    plt.plot(res1,res2)
    plt.title(f'running time for n={h} as b varies')
    plt.plot(opt_b(h),res2[opt_b(h)-bounds[0]],'x',label="b_n")
    plt.legend()
    plt.show()
   
print("Done")

## Linear Sieve Method

In [33]:
import numpy as np

def smoothness_LSM(residue,factorbase):
    f=[]
    i=0
    p=0
    while residue!=1 and i<len(factorbase):
        exponent=0
        p=ZZ(factorbase[i])
        while p.divides(residue):
            residue=residue/p           
            exponent+=1
        f.append(exponent)
        i+=1
    g=False
    if residue==1:
        g=vector(ZZ,len(factorbase))
        g[:len(f)]=f
    return g



def linear_sieve(n, s):
    #------------- set up various constants---------------------------
    L=[]
    H=ceil(sqrt(n))
    J=H^2-n

    M=ceil( exp( (0.5+s)*sqrt( ln(n)*ln(ln(n)) ) )/2 )
    
    B=exp(0.5*sqrt( ln(n)*ln(ln(n)) )  ) 
    Bi=ceil(B)

    Range=[c for c in range(-M,M+1)] 
    factorbase=[p for p in primes(Bi)]
    #--------------- for each fixed c_1, use a linear sieve------------------
    for i in range(2*M+1):  #c_1  
        c_1=Range[i]
        U=np.zeros_like(Range)  
        if Mod(H+c_1,n)!=0 and Mod(J+c_1*H,n)!=0:
            for p in primes(Bi):
    #------------- for each prime p, check wether p divides H_c_1 or not----------

                    if not p.divides(H+c_1):
                        d=-Mod(J+c_1*H,p)* ( Mod(H+c_1,p)^(-1) )
                        for j in range(i,2*M+1):  #c_2
                            if Mod(Range[j],p)==d:
                                U[j]+=log(p,2)

                    else: 
                        h_1=0
                        res=ZZ(Mod(H+c_1,n))
                        while p.divides(res):
                            h_1+=1
                            res/=p
                            
                            
                        h_2=0
                        res=ZZ(Mod(J+c_1*H,n))
                        while p.divides(res):
                            h_2+=1
                            res/=p

                        for j in range(i,2*M+1): #c_2
                            U[j]+=min(h_1,h_2)*log(p,2)
        
    
        for j in range(i,2*M+1):
            res=ZZ(Mod((H+c_1)*(H+Range[j]),n))
            if abs(U[j]-log(res,2))<=2.5 and res!=0 and gcd(res,n)==1 :
                    L.append([H+c_1, H+Range[j] ])
    return L, list(set(factorbase))

In [34]:
def Stange_kernel(n,s):   
    #-------- obtain relations-------------------------------
    List,factorbase=linear_sieve(n,s) 
#     print("Sieve Part Done")
    #-------------------- form matrix B--------------------------------
    F=[]
    rand_x=[]
    H_list=[]
    D=[]
    for L in List:
        f=smoothness_LSM(Mod(L[0]*L[1],n),factorbase)
        if f:      
            if L[0] not in H_list:
                H_list.append(L[0])
            if L[1] not in H_list:
                H_list.append(L[1])
            d=[0] * len(H_list)
            

            if L[0]==L[1]:
                d[H_list.index(L[0])]=-2
            else:
                d[H_list.index(L[0])]=-1
                d[H_list.index(L[1])]=-1
            F.append(vector(ZZ,f))
            D.append(d)

    for d in D:
        d.extend([0] * (len(H_list) - len(d)))

    DT=matrix(ZZ,D).T
    FT=matrix(ZZ,F).T
    Block=FT.stack(DT)
    print("B^T rank and dimensions",Block.rank(),Block.dimensions())
    Ext_List=factorbase+H_list
#     print("Matrix Creation Part Done")
    K_len=Block.right_nullity()
    print("nullity of M:",K_len)
    if K_len<1:            
            print("Restart with larger s")
    else:
    #------------------ check wether ker(B^T)=kern(B^s^T) for each s in S-------------------
        d=Block.dimensions()[0]
        for i in range(d):
            Ind=[ x for x in range(d)]
            Ind.remove(i)
            K_i=Block.matrix_from_rows(Ind).right_kernel().basis()
            K_i_len=len(K_i)
    #         print("index",i,"dim(K_i)=",K_i_len)
            if K_i_len!=K_len:
                print("index",i," gives matrix M_i with null(M_i)>null(M), it corresponds to the residue",Mod(Ext_List[i],n))
                if Mod(Ext_List[i],n)!=n-1 and Mod(Ext_List[i],n)!=1:
                    rand_x=vector(-Block[i,:].T)   
                    alpha=[]
                    for base_vec in K_i:
                        alp=np.dot(base_vec,rand_x)
                        alpha.append(alp)
                    order=Integer(gcd(alpha)) 
                    return order, Mod(Ext_List[i],n)
    return "No order computed", "No order of residue found"

In [35]:
n=10001

print(n)
s=0.7
order,res=Stange_kernel(n,s)
if (order,res)!= ("No order computed", "No order of residue found"):
    print("computed order",order,"of residue",res,"mod",n)
    print("actual order",Mod(res,n).multiplicative_order())
print("DONE")

10001
B^T rank and dimensions 66 (96, 110)
nullity of M: 44
index 5  gives matrix M_i with null(M_i)>null(M), it corresponds to the residue 10000
index 14  gives matrix M_i with null(M_i)>null(M), it corresponds to the residue 1
DONE


In [ ]:
for n in [231, 235]:
        print(n,"=", n.factor())          
        s=1
        order,res=Stange_kernel(n,s)
        if (order,res)!= ("No order computed", "No order of residue found"):
            print("computed order",order,"of residue",res,"mod",n)
            print("actual order",Mod(res,n).multiplicative_order())
        print("DONE")
        print()

# Section 5

## Shor's Reduction

In [ ]:
def Shor(n,b,c=10):
    for g in range(2,n-1):
        if gcd(g,n)!=1:
            return gcd(g,n),n/gcd(g,n)
        order=Stange_method(n,g,b,c)
        if order%2==0 and ZZ(Mod(g^(order/2),n))!=n-1:
                return gcd(g^(order/2)-1,n),n/gcd(g^(order/2)-1,n)


## Ekerå's Reduction

In [ ]:
def eta(q,B):
    i=1
    while q^i<=B:
        i+=1
    return i-1


def Ekera(n,b,c=10):
    g=randint(2,n-1)
    if gcd(g,n)!=1:
        return gcd(g,n),n/gcd(g,n)
    order=Stange_method(n,g,b,c)
    m=2*n.nbits()
    prod=1
    for p in prime_range(m):
        q=eta(p,m)
        prod*=p^q
        
    R=order*prod
    twopower=(int(R.binary(),2) & (~(int( (R- 1).binary() ,2)) ) )
    z=R/twopower
    
    for x in range(2,n-1):
        if gcd(x,n)!=1:
            return gcd(x,n),n/gcd(x,n)
        xz=ZZ(Mod(x,n)^z)
        d=gcd(ZZ(Mod(xz-1,n)),n)
        if d!=1:
            return d,n/d
        i=0
        while i<log(twopower,2) and xz!=1:
            xz=ZZ(Mod(xz,n)^2)
            d=gcd(xz-1,n)
            if d!=1 and d!=n:
                return d,n/d
            i+=1
            

## Comparison between two reductions

In [ ]:
from time import time
import numpy as np
import matplotlib.pyplot as plt

def running_time(f,n,b,c=10,y=50):
    summ=0
    for i in range(y):
        tock=time()
        fact1,fact2=f(n,b,c)
        if not fact1*fact2==n: 
            print("Error",n,b)
            break
        tick=time()
        summ+=tick-tock
    summ/=y
    return summ




In [ ]:
axis1=[]
axis2=[]
for n in Integers:
    b=opt_b(n)
    time1=running_time(Shor,n,b)
    axis1.append(time1)
    time2=running_time(Ekera,n,b)
    axis2.append(time2)
    

plt.plot(Integers,axis1,label="Shor's Reduction")
plt.plot(Integers,axis2,label="Ekerå's Reduction")
plt.legend()
plt.show()

# Section 6

## Single Kernel Method

In [ ]:
import numpy as np

def SK_method(n,g,b):
    order=0
    while order==0:
        #-------------------phase 1: Relation finding------------------------- 
        rank=0
        rand_x=[]                          
        F=[]   # relation Matrix

        while rank==0 :
            x=randint(1,n-1)
            while x in rand_x:
                x=randint(1,n-1)

            residue = ZZ(Mod(g,n)^x)
            f=smoothness(residue,b)

            #------------------------------Append the vector if the relationship is found---------------------
            if f:
                rand_x.append(x)
                F.append(f)  

                #--------------- Periodically compute kernel----------------------------

                rank=matrix(F,sparse=True).transpose().right_kernel().rank()

        kernel_basis=matrix(F).transpose().right_kernel().basis()
        [order]=abs(np.dot(kernel_basis,rand_x))
    return ZZ(order)


## Compare between Stange and SK Alg.

In [ ]:
def order_running_time(f,n,g,b,y=50):
    summ=0
    for i in range(y):
        tock=time()
        order=f(n,g,b)
        tick=time()
        summ+=tick-tock
    summ/=y
    return summ

In [ ]:
from time import time
import numpy as np
import matplotlib.pyplot as plt


axis1=[]
axis2=[]
Integers=[537, 765, 8585, 19053, 61453, 101371, 199989, 247251, 484957, 671015, 871933, 907423, 1744953, 1946725, 2177645, 2381625, 3632763, 5001635, 7865455, 22653803]
for n in Integers:
    b=opt_b(n)
    g=randint(2,n-1)
    while gcd(g,n)!=1:
        g=randint(2,n-1)
    time1=order_running_time(Stange_method,n,g,b,y=10)
    axis1.append(time1)
    time2=order_running_time(SK_method,n,g,b,y=10)
    axis2.append(time2)

plt.plot(Integers,axis1,label="Stange")
plt.plot(Integers,axis2,label="SK method")
plt.legend()
plt.show()

## Shor's Reduction using SK Alg.

In [ ]:
def Shor_SK(n,b):
    for g in range(2,n-1):
        if gcd(g,n)!=1:
            return gcd(g,n),n/gcd(g,n)
        order=SK_method(n,g,b)
        while order==0:
            order=SK_method(n,g,b)
            
        if order%2==0:
            M=ZZ(Mod(g^order,n))
            while M==1:
                order/=2
                M=ZZ(Mod(g^order,n))            
            if M!=n-1 and M!=1:
                return gcd(M-1,n) , n/gcd(M-1,n)
                

## Ekerå's Reduction using SK Alg.

In [ ]:
def Ekera_SK(n,b):
    g=randint(2,n-1)
    if gcd(g,n)!=1:
        return gcd(g,n),n/gcd(g,n)
    R=SK_method(n,g,b)
    twopower=(int(R.binary(),2) & (~(int( (R- 1).binary() ,2)) ) )
    z=R/twopower
    
    for x in range(2,n-1):
        if gcd(x,n)!=1:
            return gcd(x,n),n/gcd(x,n)
        b=ZZ(Mod(x,n)^z)
        
        d=gcd(ZZ(Mod(b-1,n)),n)
        if d!=1:
            return d,n/d
        i=0
        while i<log(twopower,2) and b!=1:
            b=ZZ(Mod(b,n)^2)
            d=gcd(b-1,n)
            if d!=1 and d!=n:
                return d,n/d
            i+=1

## Comparisons

In [ ]:
from time import time
import numpy as np
import matplotlib.pyplot as plt


def running_time(f,n,b,y=50):
    summ=0
    for i in range(y):
        tock=time()
        fact1,fact2=f(n,b)
        if not fact1*fact2==n: 
            break
        tick=time()
        summ+=tick-tock
    summ/=y
    return summ


In [ ]:
axis1=[]
axis2=[]
axis3=[]
axis4=[]


for n in Integers:
    b=opt_b(n)
    time1=running_time(Shor_SK,n,b)
    axis1.append(time1)
    time2=running_time(Ekera_SK,n,b)
    axis2.append(time2)
    time3=running_time(Shor,n,b)
    axis3.append(time3)
    time4=running_time(Ekera,n,b)
    axis4.append(time4)

plt.plot(Integers,axis3,label="Shor's Reduction")
plt.plot(Integers,axis1,label="Shor's Reduction using SK Alg")
plt.legend()
plt.show()

plt.plot(Integers,axis4,label="Ekerå's Reduction")
plt.plot(Integers,axis2,label="Ekerå's Reduction using SK Alg")
plt.legend()
plt.show()


## Shor's Reduction using Prop. 9

In [ ]:
def Shor_Oddorder(n,b,c=10):
    for p in [2,3,5]:
        if p.divides(n):
            return p,n/p
    for g in range(2,n-1):
        if gcd(g,n)!=1:
            return gcd(g,n),n/gcd(g,n)
        order=Stange_method(n,g,b,c)
        if order%2==0 and ZZ(Mod(g^(order/2),n))!=n-1:
                return gcd(g^(order/2)-1,n),n/gcd(g^(order/2)-1,n)
        if order%3==0 and ZZ(Mod(g^(order/3),n))!=n-1:
                return gcd(g^(order/3)-1,n),n/gcd(g^(order/3)-1,n)
        if order%5==0 and ZZ(Mod(g^(order/5),n))!=n-1:
                return gcd(g^(order/5)-1,n),n/gcd(g^(order/5)-1,n)


### Comparison

In [ ]:
Integers=[537, 765, 8585, 19053, 61453, 101371, 199989, 247251, 484957, 671015, 871933, 907423, 1744953, 1946725, 2177645, 2381625, 3632763, 5001635, 7865455, 22653803]
SOM=[86129, 112157, 187493, 802879, 1438051, 1553143, 2444393, 6044779, 33713471, 49010591]


axis1=[]
axis2=[]
i=0
for n in Integers:
    i+=1
    b=opt_b(n)
    time1=running_time(Shor,n,b)
    axis1.append(time1)
    time2=running_time(Shor_Oddorder,n,b)
    axis2.append(time2)

plt.plot(Integers,axis1,label="Shor's Reduction")
plt.plot(Integers,axis2,label="Shor's Reduction using odd oders")
plt.legend()
plt.show()

axis1=[]
axis2=[]
i=0
for n in SOM:
    i+=1
    b=opt_b(n)
    time1=running_time(Shor,n,b)
    axis1.append(time1)
    time2=running_time(Shor_Oddorder,n,b)
    axis2.append(time2)

    
plt.plot(SOM,axis1,label="Shor's Reduction")
plt.plot(SOM,axis2,label="Shor's Reduction using odd oders")
plt.legend()
plt.show()

## Jacobi Symbol Method

### Shor's Reduction with Jacobi Symbol

In [ ]:
def Shor_Jac(n,b,c=10):
    for g in range(2,n-1):
        if gcd(g,n)!=1:
            return gcd(g,n),n/gcd(g,n)

        if kronecker_symbol(g,n)==-1:
        
            order=Stange_method(n,g,b,c)
            if order%2==0 and ZZ(Mod(g^(order/2),n))!=n-1:
                    return gcd(g^(order/2)-1,n),n/gcd(g^(order/2)-1,n)


### Ekerå's Reduction with Jacobi Symbol

In [ ]:
def Ekera_Jac(n,b,c=10):
    g=randint(2,n-1)
    if gcd(g,n)!=1:
        return gcd(g,n),n/gcd(g,n)
    
    order=Stange_method(n,g,b,c)
    m=2*n.nbits()
    prod=1
    for p in prime_range(m):
        q=eta(p,m)
        prod*=p^q
        
    R=order*prod
    twopower=(int(R.binary(),2) & (~(int( (R- 1).binary() ,2)) ) )
    z=R/twopower

    for x in range(2,n-1):
        if gcd(x,n)!=1:
            return gcd(x,n),n/gcd(x,n)
        if kronecker_symbol(g,n)==-1:
            b=ZZ(Mod(x,n)^z)

            d=gcd(ZZ(Mod(b-1,n)),n)
            if d!=1:
                return d,n/d
            i=0
            while i<log(twopower,2) and b!=1:
                b=ZZ(Mod(b,n)^2)
                d=gcd(b-1,n)
                if d!=1 and d!=n:
                    return d,n/d
                i+=1
                


### Comparison

In [ ]:
Integers=[537, 765, 8585, 19053, 61453, 101371, 199989, 247251, 484957, 671015, 871933, 907423, 1744953, 1946725, 2177645, 2381625, 3632763, 5001635, 7865455, 22653803]


axis1=[]
axis2=[]
axis3=[]
axis4=[]
for n in Integers:
    b=opt_b(n)
    time1=running_time(Shor,n,b)
    axis1.append(time1)
    time2=running_time(Shor_Jac,n,b)
    axis2.append(time2)
    time3=running_time(Ekera,n,b)
    axis3.append(time3)
    time4=running_time(Ekera_Jac,n,b)
    axis4.append(time4)
    
plt.plot(Integers,axis1,label="Shor's Reduction")
plt.plot(Integers,axis2,label="Shor's Reduction using Jacobi symbol")
plt.legend()
plt.show()



plt.plot(Integers,axis3,label="Ekerå's Reduction")
plt.plot(Integers,axis4,label="Ekerå's Reduction using Jacobi symbol")
plt.legend()
plt.show()